In [ ]:
# 🎬 QHERO ClearCut v3.40 – Script para un solo video.
# Este script requiere Python 3.10.8 para funcionar correctamente.
# Fecha de creacion: 29-12-2025 22:55 v3.36.

In [2]:
# 🛠️ INSTALADOR DE LIBRERIAS MAESTRO.
# 1. Instalar librerías visuales (Soluciona "No module named rich")
# %pip install rich auto-editor deepfilternet numpy soundfile

# 2. ARREGLAR EL MOTOR DE IA (Vital para que DeepFilterNet no falle)
# Desinstalamos versiones incompatibles y ponemos las "Golden Versions"
# %pip uninstall -y torch torchaudio
# %pip install "torch==2.0.1" "torchaudio==2.0.2" --index-url https://download.pytorch.org/whl/cu118

# print("✅ INSTALACIÓN COMPLETADA. POR FAVOR, REINICIA EL KERNEL AHORA.")

In [3]:
# Onstalar IA de DeepFilterNet
# %pip install deepfilternet

In [ ]:
import subprocess
import os
import time
import sys
import shutil
import logging
from datetime import datetime
from contextlib import contextmanager
from IPython.display import clear_output

# ==============================================================================
#  🎛️ ZONA DE CONFIGURACIÓN MAESTRA (V40 SELECTIVE MASTER)
# ==============================================================================
CONFIG = {
    # --- 0. MODO DE OPERACIÓN (NUEVO) ---
    # True  = Solo corta silencios. NO toca el audio (Copia exacta). NO usa IA.
    # False = Aplica todo el tratamiento de audio (IA, EQ, Compresión) + Cortes.
    "SOLO_CORTES": False, 

    # --- 1. ARCHIVO DE ENTRADA ---
    "ARCHIVO": r"C:\Users\Nombre\Documents\31122025_1220.mp4",
    
    # --- 2. IA & LIMPIEZA INICIAL ---
    "HIGHPASS_FREQ": "115", 

    # --- 3. ECUALIZACIÓN QUIRÚRGICA (TONO) ---
    "EQ_BOX_FREQ": "350",    
    "EQ_BOX_GAIN": "-12",    
    "EQ_BOX_WIDTH": "1.5",   

    "EQ_MUD_FREQ": "250",
    "EQ_MUD_GAIN": "-4",
    "EQ_MUD_WIDTH": "1.0",

    "EQ_AIR_FREQ": "12000",
    "EQ_AIR_GAIN": "6",      
    
    # --- 4. DINÁMICA & PROCESAMIENTO ---
    "ENABLE_SUBBOOST": True, 
    "DEESSER_INTENSITY": "0.4", 
    "COMPAND_POINTS": "-90/-90 -40/-25 -20/-9 0/-3", 

    # --- 5. VOLUMEN & SILENCIO FINAL ---
    "LOUDNESS_I": "-16",     
    "LOUDNESS_TP": "-1.5",   
    "GATE_THRESHOLD": "-35dB", 
    "GATE_RELEASE": "100",     

    # --- 6. EXPORTACIÓN DE VIDEO ---
    "VIDEO_CODEC": "libx264",  
    "VIDEO_CRF": "20",         
    "VIDEO_FPS": "30",         
    "VIDEO_PRESET": "fast",    

    # --- 7. EXPORTACIÓN DE AUDIO ---
    "AUDIO_BITRATE": "320k",   
    "AUDIO_SAMPLE_RATE": "48000",

    # --- 8. EDICIÓN AUTOMÁTICA (Auto-Editor) ---
    "AUTO_CUT_MARGIN": "0.3s"  
}
# ==============================================================================

# --- LIBRERÍAS VISUALES ---
try:
    from rich.console import Console, Group
    from rich.progress import (
        Progress, SpinnerColumn, BarColumn, TextColumn, TimeElapsedColumn, TaskProgressColumn
    )
    from rich.panel import Panel
    from rich.table import Table
    from rich.theme import Theme
    from rich.live import Live
    from rich.align import Align
except ImportError:
    print("❌ Falta 'rich'. Ejecuta: %pip install rich")
    sys.exit()

# ==========================================
# 🤫 SILENCIADOR
# ==========================================
@contextmanager
def silenciar_sistema():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

# --- IMPORTAMOS IA ---
try:
    with silenciar_sistema():
        import torch
        import df.utils
        from df.enhance import enhance, init_df, load_audio, save_audio
        logging.getLogger("df").setLevel(logging.CRITICAL)
        logging.getLogger("deepfilternet").setLevel(logging.CRITICAL)
except ImportError:
    print("❌ Faltan librerías. Ejecuta: %pip install deepfilternet torch torchaudio")
    sys.exit()

# ==========================================
# 🎨 ESTÉTICA PRO
# ==========================================
COLOR_FONDO     = "#ffffff"
COLOR_TEXTO     = "#2a7d48"
COLOR_ACCENTO   = "#2596be"
COLOR_BARRA     = "#ccb623"

tema_pro = Theme({
    "base": f"{COLOR_TEXTO} on {COLOR_FONDO}",
    "accent": f"bold {COLOR_ACCENTO} on {COLOR_FONDO}",
    "dim": f"dim {COLOR_TEXTO} on {COLOR_FONDO}",
    "spinner": f"bold {COLOR_ACCENTO} on {COLOR_FONDO}",
    "bar.bg": f"grey93 on {COLOR_FONDO}",    
    "bar.complete": f"{COLOR_BARRA} on {COLOR_FONDO}",
    "bar.finished": f"{COLOR_TEXTO} on {COLOR_FONDO}",
    "percent": f"bold {COLOR_TEXTO} on {COLOR_FONDO}" 
})
console = Console(theme=tema_pro, style="base", force_jupyter=True)

# ==========================================
# 🛠️ FUNCIONES
# ==========================================

def verificar_dependencias():
    errores = []
    if not shutil.which("git"): errores.append("GIT no detectado.")
    if not shutil.which("ffmpeg"): errores.append("FFMPEG no detectado.")
    return (False, errores) if errores else (True, [])

def obtener_metadatos(archivo):
    try:
        ffprobe = shutil.which("ffprobe") or "ffprobe"
        cmd = [ffprobe, "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", archivo]
        dur = float(subprocess.check_output(cmd, creationflags=0x08000000).decode().strip())
        size_mb = os.path.getsize(archivo) / (1024 * 1024)
        return dur, f"{size_mb:.1f} MB"
    except:
        return 0, "Unknown"

def detectar_tipo_archivo(archivo):
    ext = os.path.splitext(archivo)[1].lower()
    video_exts = ['.mp4', '.mkv', '.mov', '.avi', '.webm', '.flv', '.wmv', '.m4v', '.mts', '.m2ts', '.ts', '.3gp', '.mpeg', '.mpg', '.vob', '.ogv']
    audio_exts = ['.mp3', '.wav', '.aac', '.flac', '.m4a', '.ogg', '.wma', '.opus', '.alac', '.aiff', '.pcm', '.amr']
    
    if ext in video_exts:
        return "VIDEO"
    elif ext in audio_exts:
        return "AUDIO"
    else:
        return "DESCONOCIDO"

# --- DASHBOARD DINÁMICO ---
def generar_dashboard(archivo, duracion, peso, progress_obj, estado_actual, modo):
    grid = Table.grid(expand=True, padding=(0, 2))
    grid.add_column(style="accent", justify="right", ratio=1)
    grid.add_column(style="base", justify="left", ratio=2)
    grid.add_column(style="accent", justify="right", ratio=1)
    grid.add_column(style="dim", justify="left", ratio=2)
    
    # Lógica de visualización según el modo
    if CONFIG["SOLO_CORTES"]:
        engine_desc = "[dim]⛔ SKIPPED (Cut Only Mode)[/]"
        master_desc = "[dim]⛔ SKIPPED (Original Audio)[/]"
    else:
        engine_desc = f"DeepFilterNet + V39 Custom DSP"
        master_desc = f"Gate {CONFIG['GATE_THRESHOLD']} | Cut {CONFIG['HIGHPASS_FREQ']}Hz"

    fix_desc = f"Force {CONFIG['VIDEO_FPS']}fps ({CONFIG['VIDEO_CODEC']})" if modo == "VIDEO" else "Audio Output"

    grid.add_row("MEDIA SOURCE", " ", "SETTINGS", " ")
    grid.add_row("Archivo:", os.path.basename(archivo), "Audio Engine:", engine_desc)
    grid.add_row("Duración:", f"{int(duracion//60)}m {int(duracion%60)}s", "Mastering:", master_desc)
    grid.add_row("Tamaño:", peso, "Mode:", f"{modo} ({os.path.splitext(archivo)[1]})")

    panel_top = Panel(
        grid,
        title="[accent] QHERO SUITE V40 [/]",
        subtitle="[dim] Selective Master Edition [/]",
        border_style=COLOR_ACCENTO,
        padding=(1, 2)
    )
    panel_mid = Panel(progress_obj, border_style=COLOR_ACCENTO, padding=(0, 2))
    estilo_estado = "bold" if "COMPLETADO" not in estado_actual else "bold blink"
    texto_estado = f"📢  [dim]{datetime.now().strftime('%H:%M:%S')} | INFO |[/]  [{estilo_estado}]{estado_actual}[/]"
    panel_bot = Panel(Align.center(texto_estado), border_style=COLOR_ACCENTO, padding=(0, 1))
    return Group(panel_top, panel_mid, panel_bot)

def ejecutar_comando_ffmpeg(comando_lista, progress, task):
    program = comando_lista[0]
    if program != sys.executable:
        full_path = shutil.which(program)
        if full_path: comando_lista[0] = full_path

    process = subprocess.Popen(
        comando_lista, stdout=subprocess.PIPE, stderr=subprocess.PIPE, 
        universal_newlines=True, encoding='utf-8', errors='replace', creationflags=0x08000000
    )
    stdout, stderr = process.communicate()
    if process.returncode != 0: 
        raise Exception(f"Error {program}:\n{stderr}")
    progress.advance(task, 25)

def ejecutar_ia_interna_silenciosa(input_wav, output_wav, progress, task, callback_estado):
    with silenciar_sistema():
        model, df_state, _ = init_df()
        audio, _ = load_audio(input_wav, sr=df_state.sr())
    
    chunk_sec = 60
    chunk_samples = chunk_sec * df_state.sr()
    total_samples = audio.shape[1]
    num_chunks = (total_samples + chunk_samples - 1) // chunk_samples
    
    enhanced_chunks = []
    progress.update(task, total=num_chunks, completed=0)
    
    for i in range(num_chunks):
        start = i * chunk_samples
        end = min(start + chunk_samples, total_samples)
        audio_chunk = audio[:, start:end]
        with silenciar_sistema(): enhanced_chunk = enhance(model, df_state, audio_chunk)
        enhanced_chunks.append(enhanced_chunk.cpu())
        
        pct = int(((i + 1) / num_chunks) * 100)
        progress.update(task, completed=i+1)
        callback_estado(f"IA: Limpiando Audio ({pct}%)")
        if i % 5 == 0 and torch.cuda.is_available(): torch.cuda.empty_cache()

    callback_estado("IA: Guardando archivo HQ...")
    with silenciar_sistema():
        final_audio = torch.cat(enhanced_chunks, dim=1)
        save_audio(output_wav, final_audio, int(CONFIG["AUDIO_SAMPLE_RATE"]))

# ==========================================
# 🚀 MOTOR PRINCIPAL V40 (SELECTIVE)
# ==========================================

def motor_v40_selective(archivo_entrada):
    console.clear() 
    clear_output(wait=False)
    
    # 0. VALIDACIONES
    ok, errores = verificar_dependencias()
    if not ok:
        console.print(Panel("\n".join(errores), title="❌ FALTAN REQUISITOS", border_style="red")); return
    if not os.path.exists(archivo_entrada):
        console.print(f"[bold red]❌ Archivo no encontrado[/]"); return

    # 1. PREPARACIÓN
    modo_procesamiento = detectar_tipo_archivo(archivo_entrada)
    if modo_procesamiento == "DESCONOCIDO":
        console.print(f"[bold red]❌ Formato no soportado.[/]"); return

    nombre, ext = os.path.splitext(archivo_entrada)
    ts = datetime.now().strftime("%H%M%S")
    
    # Nombres de archivos temporales
    audio_raw = f"temp_raw_{ts}.wav"
    audio_clean = f"temp_clean_{ts}.wav"
    
    if modo_procesamiento == "VIDEO":
        video_temp = f"temp_video_{ts}.mp4" 
        archivo_final = f"{nombre}_V40_Master.mp4"
    else: 
        video_temp = None 
        archivo_final = f"{nombre}_V40_Master.wav"

    duracion, peso = obtener_metadatos(archivo_entrada)

    progress = Progress(
        SpinnerColumn("dots", style="spinner"),
        BarColumn(bar_width=None, style="bar.bg", complete_style="bar.complete"), 
        TaskProgressColumn(text_format="[percent]{task.percentage:>3.0f}%"),
        TimeElapsedColumn(),
        console=console,
        expand=True
    )

    estado_global = "Inicializando..."
    def actualizar_dash(msg):
        nonlocal estado_global
        estado_global = msg
        live.update(generar_dashboard(archivo_entrada, duracion, peso, progress, estado_global, modo_procesamiento))

    with Live(generar_dashboard(archivo_entrada, duracion, peso, progress, estado_global, modo_procesamiento), console=console, refresh_per_second=5, transient=False) as live:
        task_id = progress.add_task("", total=100)
        try:
            # === RAMIFICACIÓN LÓGICA (EL INTERRUPTOR) ===
            
            if CONFIG["SOLO_CORTES"]:
                # --- MODO SOLO CORTES (SALTAR AUDIO) ---
                progress.update(task_id, completed=50) # Saltamos pasos
                
                if modo_procesamiento == "VIDEO":
                    # AUNQUE NO TOQUEMOS AUDIO, DEBEMOS ARREGLAR EL VIDEO (VFR)
                    # Si no lo hacemos, vuelve el error de la pantalla negra.
                    actualizar_dash("Modo: Solo Cortes (Estabilizando Video + Copiando Audio)...")
                    
                    cmd_bypass = [
                        "ffmpeg", "-y", "-v", "error",
                        "-i", archivo_entrada,
                        "-c:v", CONFIG["VIDEO_CODEC"], "-preset", CONFIG["VIDEO_PRESET"], "-crf", CONFIG["VIDEO_CRF"],
                        "-r", CONFIG["VIDEO_FPS"], # <--- Arregla pantalla negra
                        "-c:a", "copy",            # <--- COPIA EL AUDIO EXACTO (No lo toca)
                        video_temp
                    ]
                    ejecutar_comando_ffmpeg(cmd_bypass, progress, task_id)
                
                else:
                    # Si es audio y solo cortes, no necesitamos procesar nada intermedio,
                    # Auto-Editor puede leer el original. Pero para consistencia, copiamos.
                    shutil.copy(archivo_entrada, f"temp_bypass_{ts}{ext}")
                    video_temp = f"temp_bypass_{ts}{ext}" # Hack para usar el mismo flujo

            else:
                # --- MODO COMPLETO (IA + EQ + ETC) ---
                
                # 2. EXTRAER AUDIO
                actualizar_dash(f"FFmpeg: Extrayendo Audio ({modo_procesamiento})...")
                cmd_ext = ["ffmpeg", "-y", "-v", "error", "-i", archivo_entrada, "-vn", "-acodec", "pcm_s16le", "-ar", CONFIG["AUDIO_SAMPLE_RATE"], "-ac", "1", audio_raw]
                ejecutar_comando_ffmpeg(cmd_ext, progress, task_id)
                
                # 3. IA CLEANING
                progress.update(task_id, completed=0)
                ejecutar_ia_interna_silenciosa(audio_raw, audio_clean, progress, task_id, actualizar_dash)
                
                # 4. MASTERING
                actualizar_dash(f"Mastering: Gate {CONFIG['GATE_THRESHOLD']} + Cut {CONFIG['HIGHPASS_FREQ']}Hz...")
                progress.update(task_id, total=100, completed=50)
                
                # Filtros
                filtros = []
                if CONFIG["ENABLE_SUBBOOST"]: filtros.append("asubboost")
                filtros.append(f"highpass=f={CONFIG['HIGHPASS_FREQ']}")
                filtros.append(f"equalizer=f={CONFIG['EQ_BOX_FREQ']}:t=q:w={CONFIG['EQ_BOX_WIDTH']}:g={CONFIG['EQ_BOX_GAIN']}")
                filtros.append(f"equalizer=f={CONFIG['EQ_MUD_FREQ']}:t=q:w={CONFIG['EQ_MUD_WIDTH']}:g={CONFIG['EQ_MUD_GAIN']}")
                filtros.append(f"equalizer=f={CONFIG['EQ_AIR_FREQ']}:t=q:w=1:g={CONFIG['EQ_AIR_GAIN']}")
                filtros.append(f"deesser=i={CONFIG['DEESSER_INTENSITY']}:f=0.5:m=0.5")
                filtros.append(f"compand=attacks=0:decays=0.1:points={CONFIG['COMPAND_POINTS']}:gain=0")
                filtros.append(f"loudnorm=I={CONFIG['LOUDNESS_I']}:TP={CONFIG['LOUDNESS_TP']}:LRA=11")
                filtros.append(f"agate=threshold={CONFIG['GATE_THRESHOLD']}:ratio=10:attack=5:release={CONFIG['GATE_RELEASE']}")
                
                filtros_audio_str = ",".join(filtros)
                
                if modo_procesamiento == "VIDEO":
                    cmd_master = [
                        "ffmpeg", "-y", "-v", "error",
                        "-i", archivo_entrada, "-i", audio_clean,
                        "-c:v", CONFIG["VIDEO_CODEC"], "-preset", CONFIG["VIDEO_PRESET"], "-crf", CONFIG["VIDEO_CRF"],
                        "-r", CONFIG["VIDEO_FPS"], 
                        "-af", filtros_audio_str, 
                        "-c:a", "aac", "-b:a", CONFIG["AUDIO_BITRATE"], "-ar", CONFIG["AUDIO_SAMPLE_RATE"],
                        "-map", "0:v:0", "-map", "1:a:0",
                        "-shortest", video_temp            
                    ]
                    ejecutar_comando_ffmpeg(cmd_master, progress, task_id)
                else: 
                    cmd_master = [
                        "ffmpeg", "-y", "-v", "error",
                        "-i", audio_clean, "-af", filtros_audio_str, 
                        "-c:a", "pcm_s16le", "-ar", CONFIG["AUDIO_SAMPLE_RATE"], 
                        archivo_final            
                    ]
                    ejecutar_comando_ffmpeg(cmd_master, progress, task_id)

            # 5. CORTES FINALES (Común para ambos modos)
            if modo_procesamiento == "VIDEO" or CONFIG["SOLO_CORTES"]:
                actualizar_dash(f"Auto-Editor: Cortes (Margen {CONFIG['AUTO_CUT_MARGIN']})...")
                progress.update(task_id, completed=80)
                
                # Si estamos en modo "Solo Cortes", video_temp tiene audio original y video estabilizado
                # Si estamos en modo "Completo", video_temp tiene audio IA y video estabilizado
                cmd_cut = [sys.executable, "-m", "auto_editor", video_temp, "--margin", CONFIG['AUTO_CUT_MARGIN'], "--no-open", "-o", archivo_final]
                with silenciar_sistema():
                    subprocess.call(cmd_cut, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, creationflags=0x08000000)

            progress.update(task_id, completed=100)
            actualizar_dash("✅ PROCESO COMPLETADO")
            
        except Exception as e:
            live.stop()
            console.print(Panel(f"[bold red]❌ ERROR FATAL[/]\n\n{str(e)}", border_style="red"))
            return
        finally:
            # Limpieza
            archivos_temp = [audio_raw, audio_clean]
            if video_temp: archivos_temp.append(video_temp)
            # Limpieza extra por si usamos el hack de audio
            if f"temp_bypass_{ts}{ext}" in locals(): archivos_temp.append(f"temp_bypass_{ts}{ext}")

            for f in archivos_temp:
                if f and os.path.exists(f): os.remove(f)

    console.print(f"\n[dim]📂 Ubicación: {archivo_final}[/]")

# ==========================================
# 🏁 EJECUCIÓN
# ==========================================
if __name__ == "__main__":
    motor_v40_selective(CONFIG["ARCHIVO"])

Output()

📂 Ubicación: C:\Users\Qhero\Documents\Python\Auto-Editor\31122025_1220_V39_Master.mp4